In [23]:
import sys
sys.path.append('../src/')

from data_extraction import tools

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
import glob
import mne
import re
import os

In [40]:
path = '../data/SubjectX1/headcast_subj01_sess01_EEG_data_cube-epo.fif'
data = mne.read_epochs(path)

Reading ../data/SubjectX1/headcast_subj01_sess01_EEG_data_cube-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
327 matching events found
No baseline correction applied
327 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


In [41]:
data.info

<Info | 18 non-empty fields
    bads : list | 0 items
    ch_names : list | 1, 2, 3, 4, 5, 6, 7, 8, 9, ...
    chs : list | 60 items (EEG: 60)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    events : list | 0 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 250.0 Hz
    meas_date : NoneType | unspecified
    meas_id : dict | 4 items
    nchan : int | 60
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 500.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xplotter_layout : NoneType
>

In [28]:
test = data.get_data()

In [29]:
test.shape

(327, 60, 2001)

In [42]:
events = data.events

In [43]:
test_data = data.get_data()

In [44]:
test_data.shape

(327, 60, 2001)

In [54]:
test_data.reshape(-1, [])

(327, 60, 2001)

In [62]:
test_data.transpose(1,2,0).reshape(60, -1).shape

(60, 654327)

In [45]:
events[:10]

array([[ 1500,     0,     1],
       [ 3501,     0,     1],
       [ 5502,     0,     2],
       [ 7503,     0,     2],
       [ 9504,     0,     1],
       [11505,     0,     2],
       [13506,     0,     1],
       [15507,     0,     2],
       [17508,     0,     1],
       [19509,     0,     2]])

In [83]:
def saperate_passive_active(epoch_array):
    events = epoch_array.events
    data_array = epoch_array.get_data()
    
    data_shape = data_array.shape
    active_indexes = []
    passive_indexes = []
    for i in range(len(events)):
        if events[i][2] == 1:
            active_indexes.append(i)
        if events[i][2] == 2:
            passive_indexes.append(i)
            
    return data_array[active_indexes, :, :], data_array[passive_indexes, :, :]

def sample_data_points(data, gap=300):
    data_shape = data.shape
    sampling_indexes = [index for index in range(0, data_shape[2], gap)]
    sampled_data = data[:, :, sampling_indexes].transpose(1,2,0).reshape(data_shape[1], -1)
    return sampled_data

def create_dataset(filepaths):
    temp_active = None
    temp_passive = None
    for path in filepaths:
        epochs = mne.read_epochs(path)
        active, passive = saperate_passive_active(epochs)
        sampled_active = sample_data_points(active)
        sampled_passive = sample_data_points(passive)
        if (temp_active is not None) and (temp_passive is not None):
            temp_active = np.hstack([temp_active, sampled_active])
            temp_passive = np.hstack([temp_passive, sampled_passive])
        else:
            temp_active = sampled_active
            temp_passive = sampled_passive
        
    return temp_active, temp_passive
        
        

In [84]:
data_mat = sorted(glob.glob('../data/SubjectX1/*.fif'))
data_mat

['../data/SubjectX1\\headcast_subj01_sess01_EEG_data_cube-epo.fif',
 '../data/SubjectX1\\headcast_subj01_sess02_EEG_data_cube-epo.fif',
 '../data/SubjectX1\\headcast_subj01_sess03_EEG_data_cube-epo.fif',
 '../data/SubjectX1\\headcast_subj01_sess04_EEG_data_cube-epo.fif',
 '../data/SubjectX1\\headcast_subj01_sess05_EEG_data_cube-epo.fif',
 '../data/SubjectX1\\headcast_subj01_sess06_EEG_data_cube-epo.fif',
 '../data/SubjectX1\\headcast_subj01_sess07_EEG_data_cube-epo.fif']

In [85]:
active, passive = create_dataset(data_mat)

Reading ../data/SubjectX1\headcast_subj01_sess01_EEG_data_cube-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
327 matching events found
No baseline correction applied
327 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading ../data/SubjectX1\headcast_subj01_sess02_EEG_data_cube-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
261 matching events found
No baseline correction applied
261 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading ../data/SubjectX1\headcast_subj01_sess03_EEG_data_cube-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
347 matching events found
No baselin

In [86]:
active.shape

(60, 8260)

In [87]:
passive.shape

(60, 7728)

In [47]:
a, b = saperate_passive_active(data)

In [64]:
sampled_data = sample_data_points(a)

In [65]:
sampled_data

(60, 1197)

In [68]:
sampled_data_2 = sample_data_points(b)

In [72]:
np.hstack([sampled_data, sampled_data_2]).shape

(60, 2289)

# For Subject1

In [88]:
data_mat = sorted(glob.glob('../data/Subject1/*.fif'))
data_mat

['../data/Subject1\\subj01_sess02_EEG_data_cube_oc-epo.fif',
 '../data/Subject1\\subj01_sess03_EEG_data_cube_oc-epo.fif',
 '../data/Subject1\\subj01_sess04_EEG_data_cube_oc-epo.fif',
 '../data/Subject1\\subj01_sess05_EEG_data_cube_oc-epo.fif']

In [89]:
active, passive = create_dataset(data_mat)

Reading ../data/Subject1\subj01_sess02_EEG_data_cube_oc-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4500.00 ms
        0 CTF compensation matrices available
350 matching events found
No baseline correction applied
350 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading ../data/Subject1\subj01_sess03_EEG_data_cube_oc-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4500.00 ms
        0 CTF compensation matrices available
250 matching events found
No baseline correction applied
250 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading ../data/Subject1\subj01_sess04_EEG_data_cube_oc-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4500.00 ms
        0 CTF compensation matrices available
350 matching events found
No baseline correction applied


In [90]:
active.shape

(60, 7328)

In [91]:
passive.shape

(60, 3072)